# XGBoost models informed by feature selection analysis (using all data)

Population to train IVT decision model
Just ischaemic.
No anticolgalents
No thrombectomy patients.

SAVE FILE PATIENTS TREATED PER SCENARIO

### Plain English summary

Use a model equivalent to the one trained in notebook 040 (but for this one use all data to train the model, and no test set) to calculate the multiclass mRS distributions (individual mRS + cumulative distributions + weighted mRS) for all of the patients in the population with/without thrombolysis (Assume no-one on AFIb anticoagulant would receive thrombolysis - so remove these from the population and report the number removed).

The model includes 7 features: ["prior_disability", "stroke_severity", "stroke_team", "age", "onset-to-thrombolysis-time", "any_afib_diagnosis", "precise_onset_known"]

How would outcomes compare using:
1. actual decision made
1. benchmark
1. 'best outcome'

### Model and data
XGBoost\
7 features: ["prior_disability", "stroke_severity", "stroke_team", "age", "onset-to-thrombolysis-time", "any_afib_diagnosis", "precise_onset_known"]\
All data (not use kfolds)

### Aims

### Observations


#### Further work

#### Resources
pip install plotly
pip install dash

## Import libraries

In [186]:
# Turn warnings off to keep notebook tidy
import warnings
warnings.filterwarnings("ignore")

import copy
import os
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
#import plotly.express as px

import scipy

from xgboost import XGBClassifier
from sklearn.metrics import auc
from sklearn.metrics import roc_curve

import json

from dataclasses import dataclass

import seaborn as sns

from sklearn.metrics import roc_auc_score

from sklearn.metrics import confusion_matrix

import pickle
import shap

from os.path import exists

import math

import importlib
# Import local package
#from utils import waterfall
## Force package to be reloaded
#importlib.reload(waterfall);

# Need for cm subplots?
#from mpl_toolkits.axes_grid1 import make_axes_locatable
#from matplotlib.colors import LogNorm
#from matplotlib.ticker import MultipleLocator

#pip install category_encoders
#import category_encoders as ce

#import dash_core_components as dcc
#from dash import dcc
#import plotly.express as px
#import plotly.subplots as sp
#from plotly.offline import plot
#from plotly.subplots import make_subplots

import time

Report the time duration to run notebook

In [187]:
start_time = time.time()

In [188]:
# For those patients that do not get thrombolysis, the onset to thrombolysis time to calculate their outcome had they been given thrombolysis
ott_default = 120

Select the features for the model for disability discharge

In [189]:
selected_features_mrs = ["prior_disability", "stroke_severity", "stroke_team", 
                     "age", "onset_to_thrombolysis_time", "any_afib_diagnosis", 
                     "precise_onset_known", "discharge_disability"]

Select the features for the model for treatment

In [190]:
selected_features_treatment = ["prior_disability", "stroke_severity", 
                     "age", "arrival_to_scan_time", "precise_onset_known", 
                     "onset_to_arrival_time","onset_during_sleep", 
                     "stroke_team", "S2Thrombolysis"]

Get union of both sets of features

In [191]:
selected_features_set = list(set.union(set(selected_features_mrs), 
                                       set(selected_features_treatment)))

## Set up paths and filenames

For consistency, the folders end with "/" and the text for filenames include no trailing "_".

In [192]:
@dataclass(frozen=True)
class Paths:
    '''Singleton object for storing paths to data and database.'''
    image_save_path: str = './saved_images'
    model_save_path: str = './saved_models'
    data_save_path: str = './saved_data'
    data_read_path: str = '../data_processing/output'
    model_text: str = 'xgb_all_data_multiclass_outcome'
    notebook: str = '210_'

paths = Paths()

Create output folders if needed

In [193]:
path = paths.image_save_path
if not os.path.exists(path):
    os.makedirs(path)
        
path = paths.model_save_path
if not os.path.exists(path):
    os.makedirs(path)

path = paths.data_save_path
if not os.path.exists(path):
    os.makedirs(path)

## Import data

Data has previously been split into 5 stratified k-fold splits.

In [194]:
# Read in training set, restrict to chosen features & store
filename = os.path.join(paths.data_read_path, '02_reformatted_data_ml.csv')
data = pd.read_csv(filename)

# Create new features

### 1. Create new feature "S2Thrombolysis" from the surrogate "scan_to_thrombolysis_time"

In [195]:
data["S2Thrombolysis"] = data["scan_to_thrombolysis_time"] > -100

### 2. Create series "onset_to_thrombolysis_time_all_treated" for all patients.

To be used for the scenarios when patients that are not treated in the dataset are treated in the scenario (they are without a scan to treatment time, use the average for the hospital they attended)

First calculate the average scan_to_thrombolysis_time for those patients that got treated (per hospital) as then use this for those that do not get treatment.

In [196]:
# median scan to treatment for the treated patients (per hosptial)
mask_treatment = data["S2Thrombolysis"] == 1
median_scan_to_needle_time = (
    data[mask_treatment].groupby(["stroke_team"])["scan_to_thrombolysis_time"].median())

median_scan_to_needle_time

stroke_team
Addenbrooke's Hospital          20.0
Basildon University Hospital    34.0
Blackpool Victoria Hospital     37.0
Bradford and Airedale SU        47.0
Bronglais Hospital              39.5
                                ... 
Worthing Hospital               38.0
Wycombe General Hospital        29.0
Yeovil District Hospital        37.0
York Hospital                   29.0
Ysbyty Gwynedd                  44.0
Name: scan_to_thrombolysis_time, Length: 118, dtype: float64

Create a new series "onset_to_thrombolysis_time_all_treated" which takes the dataset value for those patients that are treated in the dataset. For those patients that are not treated in the dataset add the mdeian hospital scan to treatment time to their individual onset to scan times.

In [197]:
# Identify patients not recieve treatment in thte dataset
mask_not_treated = data["S2Thrombolysis"] == 0

# Take a deep copy of the onset to thrombolysis time. This will remain unchanged 
# for those patients that recieve treatment in the dataset
onset_to_thrombolysis_time_all_treated = data["onset_to_thrombolysis_time"].copy(deep=True)

# For those patients not recieve treatment in the dataset, use the median scan to treatment of their attended hosptial
onset_to_scan_time = data["onset_to_arrival_time"] + data["arrival_to_scan_time"]
onset_to_thrombolysis_time_all_treated[mask_not_treated] = (
    onset_to_scan_time[mask_not_treated] + 
    median_scan_to_needle_time[data["stroke_team"]].values[mask_not_treated])

onset_to_thrombolysis_time_all_treated

0          368.5
1          272.0
2          115.0
3          569.5
4          179.0
           ...  
168342     307.5
168343    1291.0
168344     368.0
168345     182.0
168346     130.0
Name: onset_to_thrombolysis_time, Length: 168347, dtype: float64

#### Select features to use in  both models

In [198]:
data = data[selected_features_set]

### One hot the categorical features

Convert some categorical features to one hot encoded features.

Define a function

In [199]:
def convert_feature_to_one_hot(df, feature_name, prefix):
    """
    df [dataframe]: training or test dataset
    feature_name [str]: feature to convert to ont hot encoding
    prefix [str]: string to use on new feature
    """

    # One hot encode a feature
    df_feature = pd.get_dummies(
        df[feature_name], prefix = prefix)
    df = pd.concat([df, df_feature], axis=1)
    df.drop(feature_name, axis=1, inplace=True)

    return(df)

Set up two lists for the one hot encoding. 

A list of the feature names that are categorical and to be converted using one hot encoding.
A list of the prefixes to use for these features.

In [200]:
features_to_one_hot = ["stroke_team", "weekday"]
list_prefix = ["team", "weekday"]

For each feature in the list, for each train and test dataset, convert to one hot encoded.

In [201]:
for feature, prefix in zip(features_to_one_hot, list_prefix):
    if feature in selected_features_set:
        data = convert_feature_to_one_hot(data, feature, prefix)

Feature names with one hot encoding

In [202]:
feature_names_ohe = list(data)

Extract the team names

In [203]:
ohe_stroke_team_features = [col for col in feature_names_ohe if col.startswith('team')]

Update the feature names to use in the model (remove "stroke_team" and add in all the one hot encoded feature names)

In [204]:
# replace the column name "stroke_team" with the ohe column names
selected_features_mrs.remove("stroke_team")
selected_features_mrs = selected_features_mrs + ohe_stroke_team_features

# replace the column name "stroke_team" with the ohe column names
selected_features_treatment.remove("stroke_team")
selected_features_treatment = selected_features_treatment + ohe_stroke_team_features

# Discharge disability outcome multiclass model

Get data for features for the outcome model

In [205]:
data_outcome = data[selected_features_mrs]

In [206]:
feature_names_ohe = list(data_outcome)
feature_names_ohe.remove("discharge_disability")
n_features_ohe = len(feature_names_ohe)

## Edit data
### Divide into X (features) and y (labels)
We will separate out our features (the data we use to make a prediction) from our label (what we are trying to predict).
By convention our features are called `X` (usually upper case to denote multiple features), and the label (disability discharge) `y`.

In [207]:
X_outcome = data_outcome.drop('discharge_disability', axis=1)
y_outcome = data_outcome['discharge_disability']

## Fit XGBoost model multiclass classification model for discharge disability

Train model

In [208]:
# Model filename
filename = os.path.join(paths.model_save_path, 
                (paths.notebook + paths.model_text + '.p'))

# Check if exists
file_exists = exists(filename)

if file_exists:
# Load models
    with open(filename, 'rb') as filehandler:
        model_outcome = pickle.load(filehandler)
else:
    # Define and Fit model
    model_outcome = XGBClassifier(verbosity=0, seed=42, learning_rate=0.5,
                            tree_method='gpu_hist')
    model_outcome.fit(X_outcome, y_outcome)

    # Save using pickle
    with open(filename, 'wb') as filehandler:
        pickle.dump(model_outcome, filehandler)

Define function to calculate the population outcome (from the individual patient mRS probabilities)

In [209]:
def calculate_population_outcome(y_probs, mrs_classes):
    weighted_mrs = (y_probs * mrs_classes).sum(axis=1)
    return(np.average(weighted_mrs),weighted_mrs)

Extract the classes from the multiclass model

In [210]:
mrs_classes = model_outcome.classes_

# Run scenarios

## 1. Scenario: All patients are treated

In [211]:
# Create X data for all treated
X_all_treated = data_outcome.drop('discharge_disability', axis=1)
X_all_treated["onset_to_thrombolysis_time"] = (
                                    onset_to_thrombolysis_time_all_treated)

# Calculate and store predicted outcome probabilities
y_outcome_probs_all_treated = model_outcome.predict_proba(X_all_treated)

# Calculate weighted outcome per patient, and for population
(ave_weighted_mrs_all_treated, weighted_mrs_all_treated) = (
        calculate_population_outcome(y_outcome_probs_all_treated, mrs_classes))

mask_all_treated = X_all_treated["onset_to_thrombolysis_time"] > -100

## 2. Scenario: No patients are treated

In [212]:
# Create X data for none treated
X_none_treated = data_outcome.drop('discharge_disability', axis=1)
X_none_treated["onset_to_thrombolysis_time"] = -100

# Calculate and store predicted outcome probabilities
y_outcome_probs_none_treated = model_outcome.predict_proba(X_none_treated)

# Calculate weighted outcome per patient, and for population
(ave_weighted_mrs_none_treated, weighted_mrs_none_treated) = (
    calculate_population_outcome(y_outcome_probs_none_treated, mrs_classes))

mask_none_treated = X_none_treated["onset_to_thrombolysis_time"] > -100

## 3. Scenario: Actual treatment decision

In [213]:
# Calculate and store predicted outcome probabilities
y_outcome_probs = model_outcome.predict_proba(X_outcome)

# Calculate weighted outcome per patient, and for population
(ave_weighted_mrs_actual_treatment, weighted_mrs_actual_treatment) = (
                    calculate_population_outcome(y_outcome_probs, mrs_classes))

mask_actual_treatment = X_outcome["onset_to_thrombolysis_time"] > -100

## 4. Scenario: Benchmark treatment decision

Read in the 25 benchmark hospitals (identified in notebook 200)

In [214]:
filename = os.path.join(paths.data_save_path, 
    ('200_xgb_10_features_all_data_thrombolysis_decision_highest_25_benchmark_'
     'hospitals_median_shap.csv'))

benchmark_hospitals = pd.read_csv(filename)
benchmark_hospitals = list(benchmark_hospitals['hospital'])

In [215]:
# Open model to get thrombolysis decision based on benchmark hospitals

# Model filename
filename_treatment_model = os.path.join(paths.model_save_path, 
                '200_xgb_10_features_all_data_thrombolysis_decision.p')

# Check if exists
file_exists = exists(filename_treatment_model)

if file_exists:
# Load models
    with open(filename_treatment_model, 'rb') as filehandler:
        model_treatment_decision = pickle.load(filehandler)
else:
    # give warning message
    print("Run notebook 200 to fit the treatment decision model")

In [226]:
# Get dataset for the treatment decision model
data_treatment_decision = data[selected_features_treatment]
X_treatment_decision = data_treatment_decision.drop('S2Thrombolysis', axis=1)

# Initialise dataframe to store benchmark hospital retults
df_benchmark_decisions = pd.DataFrame()

# For each benchmark hosptial, send all patients there, get treatment decision
for h in benchmark_hospitals:
    X_treatment_decision[ohe_stroke_team_features] = 0
    X_treatment_decision[f"team_{h}"] = 1
    df_benchmark_decisions[f"{h}"] = (
                    model_treatment_decision.predict(X_treatment_decision))

# Calculate the majority vote from the 25 benchmark hosptials
benchmark_decision = df_benchmark_decisions.sum(axis=1) > (25/2)

Use the benchmark decision whether to treat each patient

In [227]:
X_outcome_benchmark_decision = X_outcome.copy(deep=True)

# Set all patients as having thrombolysis, then set those that benchmark not 
# give as -100
X_outcome_benchmark_decision["onset_to_thrombolysis_time"] = (
                                    onset_to_thrombolysis_time_all_treated)
mask_benchmark = benchmark_decision == 0
X_outcome_benchmark_decision["onset_to_thrombolysis_time"][mask_benchmark] = -100

Calculate the population outcome for the scenario

In [228]:
# Calculate and store predicted outcome probabilities
y_outcome_probs_benchmark = model_outcome.predict_proba(
                                                X_outcome_benchmark_decision)

# Calculate weighted outcome per patient, and for population
(ave_weighted_mrs_benchmark, weighted_mrs_benchmark) = (
        calculate_population_outcome(y_outcome_probs_benchmark, mrs_classes))

## 5. Scenario: Best outcome decision

In [229]:
weighted_mrs_best_outcome = np.minimum(weighted_mrs_all_treated, 
                                   weighted_mrs_none_treated)
mask_best_outcome = weighted_mrs_all_treated < weighted_mrs_none_treated
ave_weighted_mrs_best_outcome = np.average(weighted_mrs_best_outcome)

## 6. Scenario: Worst outcome decision

In [230]:
weighted_mrs_worst_outcome = np.maximum(weighted_mrs_all_treated, 
                                    weighted_mrs_none_treated)
mask_worst_outcome = weighted_mrs_all_treated > weighted_mrs_none_treated
ave_weighted_mrs_worst_outcome = np.average(weighted_mrs_worst_outcome)

Store number of patients with indifferent outcome (in terms of weighted mRS) based on treatment

In [231]:
mask_weighted_mrs_indifferent_outcome = (
                    weighted_mrs_all_treated == weighted_mrs_none_treated)

## 7. Scenario: Best likelihood of being mRS 0-4

In [232]:
proportion_mrs0_to_4_all_treated = y_outcome_probs_all_treated[:,0:5].sum(axis=1)
proportion_mrs0_to_4_not_treated = y_outcome_probs_none_treated[:,0:5].sum(axis=1)
mask_best_proportion = (
            proportion_mrs0_to_4_all_treated > proportion_mrs0_to_4_not_treated)

weighted_mrs_best_proportion = copy.deepcopy(weighted_mrs_none_treated)
weighted_mrs_best_proportion[mask_best_proportion] = (
                            weighted_mrs_all_treated[mask_best_proportion])

ave_weighted_mrs_best_proportion = np.average(weighted_mrs_best_proportion)

## 7. Scenario: Worst likelihood of being mRS 0-4

In [233]:
mask_worst_proportion = (
            proportion_mrs0_to_4_all_treated < proportion_mrs0_to_4_not_treated)

weighted_mrs_worst_proportion = copy.deepcopy(weighted_mrs_none_treated)
weighted_mrs_worst_proportion[mask_worst_proportion] = (
                            weighted_mrs_all_treated[mask_worst_proportion])
ave_weighted_mrs_worst_proportion = np.average(weighted_mrs_worst_proportion)

Store patients with indifferent outcome (in terms of likelihood of being mRS 0 - 4) based on treatment

In [234]:
mask_weighted_mrs_indifferent_proportion = (
    proportion_mrs0_to_4_all_treated == proportion_mrs0_to_4_not_treated)

Initialise dataframe to store which patients got treatment in each scenario 

In [235]:
df_treatment_decision_per_scenario = pd.DataFrame()

df_treatment_decision_per_scenario["All_treated"] = weighted_mrs_all_treated
df_treatment_decision_per_scenario["None_treated"] = weighted_mrs_none_treated
df_treatment_decision_per_scenario["Actual_treatment_decision"] = weighted_mrs_actual_treatment
df_treatment_decision_per_scenario["Benchmark_treatment_decision"] = weighted_mrs_benchmark
df_treatment_decision_per_scenario["Best_weighted_outcome"] = weighted_mrs_best_outcome
df_treatment_decision_per_scenario["Worst_weighted_outcome"] = weighted_mrs_worst_outcome
df_treatment_decision_per_scenario["Best_proportion_outcome"] = weighted_mrs_best_proportion
df_treatment_decision_per_scenario["Worst_proportion_outcome"] = weighted_mrs_worst_proportion


In [236]:
df_treatment_decision_per_scenario.describe()

,All_treated,None_treated,Actual_treatment_decision,Benchmark_treatment_decision,Best_weighted_outcome,Worst_weighted_outcome,Best_proportion_outcome,Worst_proportion_outcome
count,168347.000000,168347.000000,168347.000000,168347.000000,168347.000000,168347.000000,168347.000000,168347.000000
mean,2.610979,2.703880,2.623641,2.589765,2.501352,2.813508,2.541487,2.773374
std,1.284881,1.368008,1.353737,1.343546,1.298883,1.338232,1.286688,1.358024
min,0.164073,0.200679,0.172511,0.172135,0.164073,0.223307,0.164073,0.172511
25%,1.577787,1.582016,1.522846,1.499427,1.448674,1.731251,1.492857,1.673468
50%,2.320550,2.368757,2.278913,2.243767,2.172313,2.502002,2.253733,2.434643
75%,3.515585,3.739424,3.605040,3.550561,3.427972,3.832023,3.443732,3.819390
max,5.947370,5.938540,5.938540,5.938540,5.938540,5.947370,5.939654,5.947370


Save results to file

In [237]:
# Model filename
filename = os.path.join(paths.data_save_path, 
                (paths.notebook + paths.model_text + '_scenario_treatment_decision_results.csv'))

df_treatment_decision_per_scenario.to_csv(filename)

Initialise dataframe to store patients mrs probabilites in each scenario 

In [238]:
df_weighted_mrs_per_scenario = pd.DataFrame()

df_weighted_mrs_per_scenario["All_treated"] = weighted_mrs_all_treated 
df_weighted_mrs_per_scenario["None_treated"] = weighted_mrs_none_treated
df_weighted_mrs_per_scenario["Actual_treatment_decision"] = weighted_mrs_actual_treatment
df_weighted_mrs_per_scenario["Benchmark_treatment_decision"] = weighted_mrs_benchmark
df_weighted_mrs_per_scenario["Best_weighted_outcome"] = weighted_mrs_best_outcome
df_weighted_mrs_per_scenario["Worst_weighted_outcome"] = weighted_mrs_worst_outcome
df_weighted_mrs_per_scenario["Best_proportion_outcome"] = weighted_mrs_best_proportion
df_weighted_mrs_per_scenario["Worst_proportion_outcome"] = weighted_mrs_worst_proportion

In [239]:
df_weighted_mrs_per_scenario.describe()

,All_treated,None_treated,Actual_treatment_decision,Benchmark_treatment_decision,Best_weighted_outcome,Worst_weighted_outcome,Best_proportion_outcome,Worst_proportion_outcome
count,168347.000000,168347.000000,168347.000000,168347.000000,168347.000000,168347.000000,168347.000000,168347.000000
mean,2.610979,2.703880,2.623641,2.589765,2.501352,2.813508,2.541487,2.773374
std,1.284881,1.368008,1.353737,1.343546,1.298883,1.338232,1.286688,1.358024
min,0.164073,0.200679,0.172511,0.172135,0.164073,0.223307,0.164073,0.172511
25%,1.577787,1.582016,1.522846,1.499427,1.448674,1.731251,1.492857,1.673468
50%,2.320550,2.368757,2.278913,2.243767,2.172313,2.502002,2.253733,2.434643
75%,3.515585,3.739424,3.605040,3.550561,3.427972,3.832023,3.443732,3.819390
max,5.947370,5.938540,5.938540,5.938540,5.938540,5.947370,5.939654,5.947370


Save results to file

In [240]:
# Model filename
filename = os.path.join(paths.data_save_path, 
    (paths.notebook + paths.model_text + '_scenario_weighted_mrs_results.csv'))

df_weighted_mrs_per_scenario.to_csv(filename)

# KP YOU ARE HERE

Most strokes are mild, they are getting the least effect from IVT. So this weights the outcome to be a smaller effect.

If look just at moderate strokes, or severe stroke, with no prior disability, this may have a bigger effect.

What is your average predicted disability w/o thrombolysis. Expect ot see a bigger effect of IVT. The worse our predicted outcome, on ave the better the effect of IVT. Largely an effect of stroke severity (0-4, 5-10, 11-15, etc... expect to see a bigger shift).

Which patients are the 126 patients with indifferent outcomes for treatment. Where's the break even point.

Interesting to look at, use a differnt decision for whether to treat. Rather than the average mRS distribution, use what if made decision based on do you improve the proportion that are less than mRS5 (0-4, vs 5-6). Do I improve the chance of not having a . Which are both improved shift, and improved reduction in being mRS5&6.

Shift most of the bulk to the left, but see an increase in mRS5&6.

Two different models, (5 and 6). Improved disability, and avoid a bad outcome.


In [241]:
mask = data["stroke_severity"] < 5
df_treatment_per_scenario_nihss_0_to_4 = df_treatment_decision_per_scenario[mask]

mask = (data["stroke_severity"] > 4 and data["stroke_severity"] < 11)
df_treatment_per_scenario_nihss_5_to_10 = df_treatment_decision_per_scenario[mask]

mask = (data["stroke_severity"] > 10 and data["stroke_severity"] < 16)
df_treatment_per_scenario_nihss_11_to_15 = df_treatment_decision_per_scenario[mask]

mask = data["stroke_severity"] > 30
df_treatment_per_scenario_nihss_30_plus = df_treatment_decision_per_scenario[mask]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

Plot (want to create a subplot per patient, with the two mRS probability distributions on the same plot)

In [ ]:
def create_df_for_bar_plot(df):
    df_bar  = pd.DataFrame()
    df_bar["Treatment"] = (["Treatment"] * 7) + (["No treatment"] * 7)
    df_bar["Discharge disability"] = np.append(df.index.values, df.index.values)
    df_bar["Probability"] = np.append(df["with"], df["without"])
    return(df_bar)

Get predictions for these patients, and store (\ dataframe for getting IVT, and a dataframe for not)

In [ ]:
list_get_treatment = ["Treatment","No treatment"]
list_onset_to_thrombolysis_time = [ott_default, -100]

list_figures = []

for patient_loc in range(n_patients):

    # take details of this patient
    df_patient_details = pd.DataFrame(columns=list(df_patient_prototypes), index=range(len(list_get_treatment)))

    # take exact copy of row
    df_patient_details.iloc[0] = df_patient_prototypes.iloc[0,:]
    df_patient_details.iloc[1] = df_patient_prototypes.iloc[0,:]

    # make first copy have, and the second copy not have thrombolysis treatment (either not get, or get at 2 hours)
    df_patient_details["onset_to_thrombolysis_time"] = list_onset_to_thrombolysis_time

    df_patient_details = df_patient_details.astype('float')

    y_probs = model.predict_proba(df_patient_details)
    y_pred = model.predict(df_patient_details)

    df_patient_mrs_results = pd.DataFrame(
            data=y_probs, 
            columns=["mRS0","mRS1","mRS2","mRS3","mRS4","mRS5","mRS6"], 
            index=["with","without"])

    df_patient_mrs_results = df_patient_mrs_results.T

    df_bar = create_df_for_bar_plot(df_patient_mrs_results)

#    ax = fig.add_subplot(n_patients, 1, patient_loc + 1)
    plotly_express_figure = px.bar(df_bar, x="Discharge disability", y="Probability", 
                color="Treatment", barmode="group")
    
    list_figures.append(plotly_express_figure)


fig = make_subplots(rows=len(list_figures), cols=1) 

for i, figure in enumerate(list_figures):
    for trace in range(len(figure["data"])):
        
        fig.append_trace(figure["data"][trace], row=i+1, col=1)

fig.update_layout(
    autosize=False,
    width=800,
    height=1600#,
#            title=f"{df_patient_prototypes.index.values[patient_loc]}",
#            xaxis_title="Discharge disability",
#            yaxis_title="Probability",
#            legend_title="Legend Title",
#            font=dict(
#                family="Courier New, monospace",
#                size=18,
#                color="RebeccaPurple"
#            )
)
#names = {'Plot 1':'2016', 'Plot 2':'2017', 'Plot 3':'2018', 'Plot 4':'2019'}

#fig.for_each_annotation(lambda a: a.update(text = df_patient_prototypes.index.values[patient_loc]))
         
fig.show()

#plot(fig)

NameError: name 'n_patients' is not defined

##### CELLS BELOW ARE CREATING A SINGLE FIGURE TO EXPERIMENT WITH THE PARAMETERS TO MAKE THE ABOVE SUBPLOT VERSION LOOK HOW WE WANT.

TO DO: 
1. Legend only have 1 entry per colour
1. Titles of the patient type for each subplot
1. X and Y axis labels

In [ ]:
list_get_treatment = ["Treatment","No treatment"]
list_onset_to_thrombolysis_time = [ott_default, -100]

# take details of this patient
df_patient_details = pd.DataFrame(columns=list(df_patient_prototypes), index=range(len(list_get_treatment)))

# take exact copy of row
df_patient_details.iloc[0] = df_patient_prototypes.iloc[0,:]
df_patient_details.iloc[1] = df_patient_prototypes.iloc[0,:]

# make first copy have, and the second copy not have thrombolysis treatment (either not get, or get at 2 hours)
df_patient_details["onset_to_thrombolysis_time"] = list_onset_to_thrombolysis_time

df_patient_details = df_patient_details.astype('float')

df_patient_details


In [ ]:
y_probs = model.predict_proba(df_patient_details)
y_pred = model.predict(df_patient_details)

In [ ]:
df_patient_mrs_results = pd.DataFrame(
            data=y_probs, 
            columns=["mRS0","mRS1","mRS2","mRS3","mRS4","mRS5","mRS6"], 
            index=["with","without"])

In [ ]:
df_patient_mrs_results = df_patient_mrs_results.T
df_patient_mrs_results

In [ ]:
df_bar = create_df_for_bar_plot(df_patient_mrs_results)
df_bar

In [ ]:
px.bar(df_bar, x="Discharge disability", y="Probability", 
                color="Treatment", barmode="group")# title=f"{df_patient_details.index.value}")